### Задание по программированию: 1NN против RandomForest

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#vibo: загружаем датасет load_digits
digits = datasets.load_digits()

#vibo: ключи датасета
digits.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])

In [2]:
#vibo: задаем признаки
X = digits.data
#vibo: задаем метки правильных ответов
y = digits.target

In [3]:
#vibo: выделяем подвыборки для обучения и контроля качества (на тест последние 25% по улсловию задачи)
train_size = 0.75
train_size_number = int(y.shape[0] * train_size)
X_train = X[:train_size_number]
X_test = X[train_size_number:]
y_train = y[:train_size_number]
y_test = y[train_size_number:]

#### Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [4]:
def euclidian_metric(x, y):
    return np.linalg.norm(x - y)

In [5]:
y_knn = []
for test_value in X_test:
    ind_min_metric = 0
    min_metric = euclidian_metric(test_value, X_train[0])
    
    for index, train_value in enumerate(X_train):
        metric = euclidian_metric(test_value, train_value)
        if metric < min_metric:
            min_metric = metric
            ind_min_metric = index
            
    y_knn.append(y_train[ind_min_metric])

In [6]:
knn_err_rate = 1 - accuracy_score(y_test, y_knn)
knn_err_rate

0.0377777777777778

In [7]:
def write_answer(answer, filename):
    with open(filename, 'w') as fout:
        fout.write(str(answer))

write_answer(knn_err_rate, "ans1.txt")

#### vibo: Итого задание 1. Доля ошибок kNN = 0.0378. (ответ анализатора:  (Верно. Обратите внимание - ошибка составляет менее 5%! Это показательный пример, что никогда не стоит забывать о бейзлайнах - иногда легко податься соблазну запустить несколько сложных алгоритмов, получить в них качество 90+%, и верить в то, что задача решена хорошо. При этом же запросто может оказаться, что простые методы дают еще лучший результат.)

#### Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [8]:
estimator_2 = ensemble.RandomForestClassifier(n_estimators=1000)

cross_val_score_2 = cross_val_score(estimator_2, X_test, y_test, scoring='accuracy').mean()
1 - cross_val_score_2

0.053333333333333344

In [9]:
write_answer(1 - cross_val_score_2, "ans2.txt")

#### vibo: Итого задание 2. Доля ошибок RandomForest = 0.0555. (ответ анализатора: обратите внимание, что качество заметно хуже, чем у 1NN. Это нестандартная ситуация, как правило Random Forest отрабатывает лучше, но такие ситуации иногда встречаются.).